# Replace Manual version of Logistic Regression with TF based version.

## Import functions and data

In [38]:
import nltk
from nltk.corpus import twitter_samples 
import pandas as pd

import tensorflow as tf

In [39]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to C:\Users\Utsav
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Utsav
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [41]:
#process_tweet(): cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets


    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
            
            #############################################################
            # 1 remove stopwords
            # 2 remove punctuation
            # 3 stemming word
            # 4 Add it to tweets_clean
            if (word not in stopwords_english and word not in string.punctuation):
                stem_word = stemmer.stem(word)
                tweets_clean.append(stem_word)
        

    return tweets_clean

In [42]:
#build_freqs counts how often a word in the 'corpus' (the entire set of tweets) was associated with
  # a positive label '1'         or 
  # a negative label '0', 

#then builds the freqs dictionary, where each key is a (word,label) tuple, 

#and the value is the count of its frequency within the corpus of tweets.

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            
            #############################################################
            #Update the count of pair if present, set it to 1 otherwise
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
            

    return freqs

### Prepare the data
* The `twitter_samples` contains subsets of 5,000 positive tweets, 5,000 negative tweets, and the full set of 10,000 tweets.  

In [43]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [44]:
# split the data into two pieces, one for training and one for testing
#############################################################
from sklearn.model_selection import train_test_split

train_pos, test_pos = train_test_split(all_positive_tweets, test_size=0.2)
train_neg, test_neg = train_test_split(all_negative_tweets, test_size=0.2)

train_x = train_pos + train_neg
test_x = test_pos + test_neg

* Create the numpy array of positive labels and negative labels.

In [45]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

* Create the frequency dictionary using the  `build_freqs()` function.  
    


In [46]:
# create frequency dictionary
#############################################################
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11329


* HERE, The `freqs` dictionary is the frequency dictionary that's being built. 
* The key is the tuple (word, label), such as ("happy",1) or ("happy",0).  The value stored for each key is the count of how many times the word "happy" was associated with a positive label, or how many times "happy" was associated with a negative label.

### Process tweet

In [47]:
# Example
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 Final few days in #Hale #Cheshire, got some lovely before and after shots which I'll be sharing tomorrow. Call to book #decorating :)

This is an example of the processed version of the tweet: 
 ['final', 'day', 'hale', 'cheshir', 'got', 'love', 'shot', "i'll", 'share', 'tomorrow', 'call', 'book', 'decor', ':)']


## Extracting the features

* Given a list of tweets, extract the features and store them in a matrix. You will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet. 
* Then train your logistic regression classifier on these features.
* Test the classifier on a validation set. 


In [48]:
def extract_features(tweet, freqs):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # tokenizes, stems, and removes stopwords
    #############################################################
    output = []
    for word_l in tweet:
        word_l = process_tweet(word_l)

        # 3 elements in the form of a 1 x 3 vector
        x = np.zeros((1, 3))

        #bias term is set to 1
        x[0,0] = 1

        # loop through each word in the list of words
        for word in word_l:

            # increment the word count for the positive label 1
            x[0,1] += freqs.get((word, 1.0),0)

            # increment the word count for the negative label 0
            x[0,2] += freqs.get((word, 0.0),0)


        assert(x.shape == (1, 3))
        output.append(x)
    return output

In [49]:
final_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [50]:
final_model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
final_model.fit(tf.convert_to_tensor(extract_features(train_x, freqs)), train_y, epochs=5)

Epoch 1/5
250/250 [==============================] - 0s 645us/step - loss: 49.2958 - accuracy: 0.9896
Epoch 2/5
250/250 [==============================] - 0s 688us/step - loss: 34.3933 - accuracy: 0.9949
Epoch 3/5
250/250 [==============================] - 0s 649us/step - loss: 28.8591 - accuracy: 0.9939
Epoch 4/5
250/250 [==============================] - 0s 657us/step - loss: 23.1159 - accuracy: 0.9941
Epoch 5/5
250/250 [==============================] - 0s 631us/step - loss: 17.6928 - accuracy: 0.9940


In [52]:
final_model.evaluate(tf.convert_to_tensor(extract_features(test_x, freqs)), test_y)

63/63 [==============================] - 0s 660us/step - loss: 6.6790 - accuracy: 0.9895


[6.678951263427734, 0.9894999861717224]